In [ ]:
ENV_NAME = "MountainCar"

In [ ]:
!pip install stable-baselines3[extra] imageio

In [ ]:
# import libraries
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import imageio
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback, BaseCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from IPython.display import Image, display,clear_output

from gymnasium.wrappers import TimeLimit
import os

In [ ]:
# Lookup which scenarios in gymnasium are well suited for DQN (linear and lowdimensional (2D))

for env_spec in gym.registry.values():
    if hasattr(env_spec, "entry_point"):
        try:
            env = gym.make(env_spec.id)
            if isinstance(env.action_space, gym.spaces.Discrete):
                print(env_spec.id)
        except:
            continue

In [ ]:
# The structure looks something like this:
"""
DummyVecEnv
└── Monitor
    └── TimeLimit
        └── CartPoleEnv
"""

#We need to umwrap the structure until we reach the instance of monitor
def unwrap_monitor(env):
    """Recursively unwrap env until we find a Monitor."""
    current = env
    while not isinstance(current, Monitor):
        if hasattr(current, 'env'):
            current = current.env
        else:
            raise ValueError("Monitor wrapper not found in the environment stack.")
    return current

# ✅ A Gym environment → wrapped in a Monitor(logs metrics, rewards... for episodes) → then wrapped in a VecEnv (like DummyVecEnv)
def make_env():
    def _init():
        env = gym.make(ENV_NAME, render_mode="rgb_array")
        env = RewardShapingWrapper(env)
        env = Monitor(env)  # Wrap with Monitor first
        return env
    return DummyVecEnv([_init])

class RewardShapingWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        position = obs[0]
        shaped_reward = reward + abs(position + 0.5)  # Encourage movement far from default positions
        return obs, shaped_reward, terminated, truncated, info


class LiveRenderCallback(BaseCallback):
    def __init__(self, render_freq=1000, verbose=0):
        super().__init__(verbose)
        self.render_freq = render_freq

    def _on_step(self) -> bool:
        if self.n_calls % self.render_freq == 0:
            shot = self.training_env.render(mode="rgb_array")
            clear_output(wait=True)
            plt.imshow(shot)
            plt.axis("off")
            plt.show()
        return True


In [ ]:
# Create and wrap environment (Monitor helps log rewards)

env = make_env()

#env = DummyVecEnv([lambda: Monitor(gym.make("CartPole-v1", render_mode="rgb_array"))])


# Create eval environment (for callback evaluation)
eval_env = make_env()

# Create directory to save models and videos
os.makedirs("models", exist_ok=True)
os.makedirs("videos", exist_ok=True)

# Create the DQN agent
model = DQN(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=1e-3,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=32,
    gamma=0.99,
    train_freq=4,
    target_update_interval=1000,
    exploration_fraction=0.1,
    exploration_final_eps=0.02,
    tensorboard_log="./dqn_cartpole_tensorboard/",
    seed=42,
)

# Setup evaluation callback to save best model during training
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./models/best_model",
    log_path="./models/",
    eval_freq=5000,
    deterministic=True,
    render=False,
)

# Train the model (you can increase total_timesteps for better results)
model.learn(total_timesteps=100000, callback=[eval_callback, LiveRenderCallback()])

# Save the final model
model.save("models/dqn_cartpole_final")

# Load best saved model
model = DQN.load("./models/best_model/best_model.zip", env=env)

# Run one episode and record frames for GIF
obs = env.reset()
done = False
frames = []
done = False
while not done:
    frame = env.render()
    frames.append(frame)
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)

# Save GIF
gif_path = "videos/cartpole_agent.gif"
imageio.mimsave(gif_path, frames, fps=30)

# Display GIF inline (Jupyter/Colab)
display(Image(filename=gif_path))

# Plot episode rewards logged during training
monitor_env = unwrap_monitor(env.envs[0]) #Unwraps the structure of the first environment [0] until it finds the monitor
results = monitor_env.get_episode_rewards()

env.close()

plt.figure(figsize=(12, 6))
plt.plot(results)
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Training Rewards per Episode")
plt.show()

# Instructions to run tensorboard (if desired):
# In terminal: tensorboard --logdir ./dqn_cartpole_tensorboard/


In [ ]:
"""
+-------------+ (step)   +-----------------+          +--------------------+
|             | action   |                 | new obs  |                    |
|   Agent     +--------->+   Environment   +--------->+     Agent again    |
|  (predict)  |          |                 | reward   |                    |
+-------------+          +-----------------+  done?   +--------------------+
                                  ^                               |
                                  |<-------------[loop]-----------+

"""

## Hyperparameter tuning with manual search

import itertools

def train_dqn_with_params(env, params):
    model = DQN(
        "MlpPolicy",
        env,
        learning_rate=params["learning_rate"],
        batch_size=params["batch_size"],
        gamma=params["gamma"],
        verbose=0,
    )
    model.learn(total_timesteps=10000)
    return model

search_space = {
    "learning_rate": [1e-3, 5e-4],
    "batch_size": [32, 64],
    "gamma": [0.95, 0.99]
}

def evaluate_model(model, env, n_episodes=5):
    rewards = []
    for _ in range(n_episodes):
        obs = env.reset()
        done = False
        total_reward = 0
        while not done:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            total_reward += reward
        rewards.append(total_reward)
    return sum(rewards) / n_episodes

best_params = None
best_reward = -float('inf')
results = []

for values in itertools.product(*search_space.values()):
    params = dict(zip(search_space.keys(), values))
    print(f"Training with params: {params}")
    model = train_dqn_with_params(env, params)
    avg_reward = evaluate_model(model, env)
    print(f"Average reward: {avg_reward}")

    results.append((params, avg_reward))
    if avg_reward > best_reward:
        best_reward = avg_reward
        best_params = params

print(f"Best params: {best_params} with average reward {best_reward}")

In [ ]:
## Hyperparameter tuning with random search

import random

def random_search(env, search_space, n_trials=10):
    results = []
    for _ in range(n_trials):
        params = {k: random.choice(v) for k, v in search_space.items()}
        print(f"Training with params: {params}")
        model = train_dqn_with_params(env, params)
        # Evaluate model here
        results.append((params, model))
    return results

# Use same search_space as above
results = random_search(env, search_space)


In [ ]:
!pip install optuna

In [ ]:
## Hyperparameter tuning automated with optuna

import optuna
from stable_baselines3.common.evaluation import evaluate_policy

def objective(trial):
    params = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-3),
        "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128]),
        "gamma": trial.suggest_uniform("gamma", 0.9, 0.999),
    }
    model = DQN("MlpPolicy", env, **params, verbose=0)
    model.learn(total_timesteps=10000)
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
    return mean_reward

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best params:", study.best_params)
